In [21]:
import pandas as pd 

In [22]:
import numpy as np

In [23]:
!ls ../out

output_CM_global.csv	output_SLATM_global.csv
output_FCHL_global.csv	output_SOAP_global.csv


In [24]:
slatm_out = pd.read_csv("../out/output_FCHL_global.csv", index_col=0)

In [25]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen
0,1,"['frag_02_c00001', 'frag_16_c00001']",3.304072,3.304072
1,2,"['frag_05_c00001', 'frag_16_c00001']",2.513089,3.513089
2,3,"['frag_04_c00001', 'frag_16_c00001']",3.579382,3.579382
3,4,"['frag_02_c00001', 'frag_04_c00001', 'frag_16_...",1.663799,3.663799
4,5,"['frag_08_c00001', 'frag_16_c00001']",1.702805,3.702805
...,...,...,...,...
995,996,"['frag_03_c00001', 'frag_04_c00001', 'frag_08_...",9.175219,15.175219
996,997,"['frag_06_c00001', 'frag_08_c00001', 'frag_09_...",9.179986,15.179986
997,998,"['frag_02_c00001', 'frag_06_c00001', 'frag_08_...",9.185319,15.185319
998,999,"['frag_01_c00001', 'frag_02_c00001', 'frag_06_...",10.197514,15.197514


In [26]:
from ast import literal_eval 

In [27]:
slatm_out["Fragments"] = slatm_out["Fragments"].apply(literal_eval)

In [28]:
# excess is missing Hs - to fix 

In [29]:
import qml

In [30]:
qm9_mol = qml.Compound("../targets/qm9_0.xyz")
qm9_ncharges = qm9_mol.nuclear_charges

In [31]:
qm9_ncharges

array([8, 6, 6, 7, 6, 8, 8, 7, 6, 1, 1])

In [32]:
def count_ncharges(list_ncharges, elements=[1,6,7,8,16]):
    ncharge_count = np.zeros(len(elements))
    for ncharge in list_ncharges:
        for i, elem in enumerate(elements):
            if ncharge == elem:
                ncharge_count[i] += 1
    return ncharge_count

In [33]:
qm9_counts = count_ncharges(qm9_ncharges)

In [34]:
qm9_counts

array([2., 4., 2., 3., 0.])

In [35]:
def get_excess_frags(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    diff_count = np.array(diff_count).astype(int)
    excess = []
    for i, count in enumerate(diff_count): 
        ncharge = [1,6,7,8,16][i]
        if count > 0 :
            for i in range(count):
                excess.append(ncharge)
        elif count < 0:
            for i in range(abs(count)):
                excess.append(-ncharge)
    return excess

In [36]:
slatm_out["Excess"] = slatm_out["Fragments"].apply(get_excess_frags)

In [37]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_02_c00001, frag_16_c00001]",3.304072,3.304072,"[1, 1, 1, 1, 6, -8]"
1,2,"[frag_05_c00001, frag_16_c00001]",2.513089,3.513089,"[1, 1, 8]"
2,3,"[frag_04_c00001, frag_16_c00001]",3.579382,3.579382,"[1, 1]"
3,4,"[frag_02_c00001, frag_04_c00001, frag_16_c00001]",1.663799,3.663799,"[1, 1, 1, 1, 1, 1, 6, 6]"
4,5,"[frag_08_c00001, frag_16_c00001]",1.702805,3.702805,"[1, 1, 1, 6, 7]"
...,...,...,...,...,...
995,996,"[frag_03_c00001, frag_04_c00001, frag_08_c0000...",9.175219,15.175219,"[1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 7, 7, 8]"
996,997,"[frag_06_c00001, frag_08_c00001, frag_09_c0000...",9.179986,15.179986,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, ..."
997,998,"[frag_02_c00001, frag_06_c00001, frag_08_c0000...",9.185319,15.185319,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, ..."
998,999,"[frag_01_c00001, frag_02_c00001, frag_06_c0000...",10.197514,15.197514,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, ..."


In [38]:
slatm_out.sort_values(by='ObjValWithPen').iloc[0:50]

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_02_c00001, frag_16_c00001]",3.304072,3.304072,"[1, 1, 1, 1, 6, -8]"
1,2,"[frag_05_c00001, frag_16_c00001]",2.513089,3.513089,"[1, 1, 8]"
2,3,"[frag_04_c00001, frag_16_c00001]",3.579382,3.579382,"[1, 1]"
3,4,"[frag_02_c00001, frag_04_c00001, frag_16_c00001]",1.663799,3.663799,"[1, 1, 1, 1, 1, 1, 6, 6]"
4,5,"[frag_08_c00001, frag_16_c00001]",1.702805,3.702805,"[1, 1, 1, 6, 7]"
5,6,"[frag_12_c00001, frag_13_c00001]",1.863887,3.863887,"[1, 1, 1, 1, 1, 6, 7]"
6,7,"[frag_11_c00001, frag_13_c00001]",2.295007,4.295007,"[1, 1, 1, 1, 1, 6, 7]"
7,8,"[frag_01_c00001, frag_02_c00001, frag_16_c00001]",3.304072,4.304072,"[1, 1, 1, 1, 1, 1, 6]"
8,9,"[frag_09_c00001, frag_16_c00001]",2.478475,4.478475,"[1, 1, 1, 1, 6, 6]"
9,10,"[frag_01_c00001, frag_05_c00001, frag_16_c00001]",2.513089,4.513089,"[1, 1, 1, 1, 8, 8]"
